In [ ]:
#Example on applying LoRA to BERT Model
from lora import LoRALayer
import torch
from torch import nn
import numpy as np
import tensorflow as tf
from transformers import BertForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets
import os
import pandas as pd


2025-03-02 21:00:57.714276: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-02 21:00:58.095826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740949258.253285    1966 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740949258.294645    1966 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 21:00:58.594522: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [5]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceCl

In [6]:
model.bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
#Freezes all parameters of model before applying LoRA
for param in model.parameters():
    param.requires_grad = False

In [9]:
rank = 8
alpha = 16

In [ ]:
#Applies LoRA to all attention layers
def apply_lora(rank, alpha):
    for idx, layer in enumerate(model.bert.encoder.layer):
        query = model.bert.encoder.layer[idx].attention.self.query
        key = model.bert.encoder.layer[idx].attention.self.key
        value = model.bert.encoder.layer[idx].attention.self.value
        model.bert.encoder.layer[idx].attention.self.query = LoRALayer(query, rank, alpha)
        model.bert.encoder.layer[idx].attention.self.key = LoRALayer(key, rank, alpha)
        model.bert.encoder.layer[idx].attention.self.value = LoRALayer(value, rank, alpha)

In [ ]:
apply_lora(rank, alpha)

In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [12]:
ds = load_dataset("stanfordnlp/imdb")

In [13]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [15]:
print(ds["train"])
print(pd.DataFrame(ds["train"]))
print(ds["train"])
print(pd.DataFrame(ds["train"]))

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})
                                                    text  label
0      I rented I AM CURIOUS-YELLOW from my video sto...      0
1      "I Am Curious: Yellow" is a risible and preten...      0
2      If only to avoid making this type of film in t...      0
3      This film was probably inspired by Godard's Ma...      0
4      Oh, brother...after hearing about this ridicul...      0
...                                                  ...    ...
24995  A hit at the time but now better categorised a...      1
24996  I love this movie like no other. Another time ...      1
24997  This film and it's sequel Barry Mckenzie holds...      1
24998  'The Adventures Of Barry McKenzie' started lif...      1
24999  The story centers around Barry McKenzie who mu...      1

[25000 rows x 2 columns]
Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})
                                                    text  label
0      I r

In [16]:
def tokenize(example):
    inputs = bert_tokenizer(
        example["text"],
        padding=True,
        truncation=True,
        return_tensors="np",
        max_length=512
    )
    return inputs

In [17]:
train_ds = ds.map(tokenize, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [18]:
finalized_ds = train_ds.copy()

In [19]:
finalized_ds["train"] = finalized_ds["train"].remove_columns(["text", "token_type_ids"]).select(range(5000))

In [20]:
finalized_ds["test"] = finalized_ds["test"].remove_columns(["text", "token_type_ids"]).select(range(1000))

In [27]:
lr = 2e-4
epochs = 10


In [32]:
training_args = TrainingArguments(
    output_dir="lora_tuned.pt",
    num_train_epochs=epochs,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    report_to="none",
    learning_rate=lr,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_strategy="epoch"

)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=finalized_ds["train"],
    eval_dataset=finalized_ds["test"]

)

In [34]:
trainer.train()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.029600,0.019235
2,0.025000,0.017476
3,0.024200,0.016581
4,0.022800,0.016745
5,0.022000,0.015049
6,0.021200,0.014866
7,0.020200,0.014431
8,0.019600,0.014107
9,0.018900,0.013660
10,0.018700,0.013450


TrainOutput(global_step=3130, training_loss=0.022226985346394987, metrics={'train_runtime': 4237.389, 'train_samples_per_second': 11.8, 'train_steps_per_second': 0.739, 'total_flos': 1.32235004928e+16, 'train_loss': 0.022226985346394987, 'epoch': 10.0})